# Goal

The objective of this notebook is to demonstrate and visualize the computation of the pareto frontier.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import matplotlib.pyplot as plt
import pandas as pd

from mlos.Optimizers.OptimizationProblem import OptimizationProblem, Objective
from mlos.Optimizers.ParetoFrontier import ParetoFrontier
from mlos.Spaces import SimpleHypergrid, ContinuousDimension


Let's put toghether a very simple set of points, two of which are on the frontier, one is technically on the frontier, but is dominated by the first two, and several other points are strictly dominated.

Assuming we are trying to maximize both y1 and y2:
* Pareto efficient points (y1,y2): (1,2), (2,1)
* Points on the frontier line but dominated: (0.5, 2), (1,1), (2, 0.5)
* Strictly dominated points: (0.5, 0.5), (0.5, 1), (0.5, 1.5), (1, 0.5), (1.5, 0.5)

In [4]:
optimization_problem = OptimizationProblem(
    parameter_space=None,
    objective_space=SimpleHypergrid(
        name="objectives",
        dimensions=[
            ContinuousDimension(name='y1', min=0, max=5),
            ContinuousDimension(name='y2', min=0, max=5)
        ]
    ),
    objectives=[
        Objective(name='y1', minimize=False),
        Objective(name='y2', minimize=False)
    ]
)

objectives_df = pd.DataFrame(
    [
        [1, 2],
        [2, 1],
        [0.5, 2],
        [1, 1],
        [2, 0.5],
        [0.5, 0.5],
        [0.5, 1],
        [0.5, 1.5],
        [1, 0.5],
        [1.5, 0.5]
    ],
    columns=['y1', 'y2']
)

pareto_line = pd.DataFrame(
    [
        [0.5, 2],
        [1, 2],
        [1, 1],
        [2, 1],
        [2, 0.5],
    ],
    columns=['y1', 'y2']
)

parameters_df = pd.DataFrame(index=objectives_df.index)

pareto_frontier = ParetoFrontier(optimization_problem, objectives_df, parameters_df)
pareto_df = pareto_frontier.pareto_df

plt.figure(figsize=(5, 5))
plt.scatter(objectives_df['y1'], objectives_df['y2'], label="objectives")
plt.scatter(pareto_df['y1'], pareto_df['y2'], label="pareto")
plt.plot(pareto_line['y1'], pareto_line['y2'], label="frontier line")
plt.legend()
plt.ylabel('y2')
plt.xlabel('y1')

Text(0.5, 0, 'y1')

In [5]:
# Let's see if we can determine if a point is dominated or not.
#
dominated_points_df = pd.DataFrame({
    'y1': [0.8, 0.8, 1.6],
    'y2': [0.8, 1.6, 0.8]
})

non_dominated_points_df = pd.DataFrame({
    'y1': [0.8, 1.6, 2.6],
    'y2': [2.6, 1.6, 0.8]
})

plt.figure(figsize=(5, 5))
plt.scatter(objectives_df['y1'], objectives_df['y2'], label="objectives")
plt.scatter(pareto_df['y1'], pareto_df['y2'], label="pareto")
plt.scatter(dominated_points_df['y1'], dominated_points_df['y2'], label='dominated')
plt.scatter(non_dominated_points_df['y1'], non_dominated_points_df['y2'], label='non dominated')
plt.plot(pareto_line['y1'], pareto_line['y2'], label="frontier line")
plt.legend()
plt.ylabel('y2')
plt.xlabel('y1')

Text(0.5, 0, 'y1')

In [6]:
dominated_points_df.apply(lambda row: (row >= pareto_df).any(axis=1)).any(axis=1)


1    False
4    False
0    False
3    False
2    False
dtype: bool

In [7]:
non_dominated_points_df.apply(lambda row: (row >= pareto_df).any(axis=0))

,y1,y2
y1,False,False
y2,False,False
0,False,False
1,False,False
2,False,False


In [8]:
# dominated means that at least one point in the pareto is larger than the candidate point in every way.
def is_not_dominated(objectives_df, pareto_df):
    is_not_dominated = pd.Series([True for i in range(len(objectives_df.index))])
    for idx, pareto_row in pareto_df.iterrows():
        is_not_dominated = is_not_dominated & (objectives_df >= pareto_row).any(axis=1)
    return is_not_dominated

is_not_dominated(non_dominated_points_df, pareto_df)

0    True
1    True
2    True
dtype: bool

In [9]:
is_not_dominated(dominated_points_df, pareto_df)

0    False
1    False
2    False
dtype: bool

In [10]:
mixed = pd.concat([dominated_points_df, non_dominated_points_df], axis=0, ignore_index=True)
mixed

,y1,y2
0,0.8,0.8
1,0.8,1.6
2,1.6,0.8
3,0.8,2.6
4,1.6,1.6
5,2.6,0.8


In [11]:
assert not pareto_frontier.is_dominated(non_dominated_points_df).any()

In [12]:
assert pareto_frontier.is_dominated(dominated_points_df).all()

In [13]:
assert pareto_frontier.is_dominated(mixed).sum() == 3

In [15]:
from mlos.Optimizers.BayesianOptimizerFactory import BayesianOptimizerFactory, bayesian_optimizer_config_store
from mlos.OptimizerEvaluationTools.ObjectiveFunctionFactory import ObjectiveFunctionFactory
from mlos.OptimizerEvaluationTools.SyntheticFunctions.Hypersphere import Hypersphere
from mlos.Spaces import ContinuousDimension, DiscreteDimension, Point, SimpleHypergrid

hypersphere_radius = 10

objective_function_config = Point(
    implementation=Hypersphere.__name__,
    hypersphere_config=Point(
        num_objectives=2,
        minimize='none',
        radius=hypersphere_radius
    )
)

objective_function = ObjectiveFunctionFactory.create_objective_function(objective_function_config=objective_function_config)
optimization_problem = objective_function.default_optimization_problem
random_params_df = optimization_problem.parameter_space.random_dataframe(100)

# Let's randomly subsample 10% of points in random_params_df and make those points pareto optimal.
#
optimal_points_index = random_params_df.sample(
    frac=0.1,
    replace=False,
    axis='index'
).index

random_params_df.loc[optimal_points_index, ['radius']] = hypersphere_radius


dominated_params_df = random_params_df.loc[optimal_points_index].copy()
dominated_params_df['radius'] = 1
dominated_points_df = objective_function.evaluate_dataframe(dataframe=dominated_params_df)

non_dominated_params_df = random_params_df.loc[optimal_points_index].copy()
non_dominated_params_df['radius'] = 11
non_dominated_points_df = objective_function.evaluate_dataframe(dataframe=non_dominated_params_df)


objectives_df = objective_function.evaluate_dataframe(dataframe=random_params_df)
pareto_frontier = ParetoFrontier(
    optimization_problem=optimization_problem,
    objectives_df=objectives_df,
    parameters_df=pd.DataFrame(index=objectives_df.index)
)
pareto_df = pareto_frontier.pareto_df

In [16]:
assert pareto_frontier.is_dominated(dominated_points_df).all()

In [17]:
assert not pareto_frontier.is_dominated(non_dominated_points_df).any()

In [20]:
plt.figure(figsize=(5, 5))
#plt.scatter(objectives_df['y0'], objectives_df['y1'], label="objectives")
plt.scatter(pareto_df['y0'], pareto_df['y1'], label="pareto")
plt.scatter(dominated_points_df['y0'], dominated_points_df['y1'], label='dominated')
plt.scatter(non_dominated_points_df['y0'], non_dominated_points_df['y1'], label='non dominated')
plt.legend()
plt.ylabel('y1')
plt.xlabel('y0')

Text(0.5, 0, 'y0')